In [5]:
import glob
import pprint
import re
import os

import lxml.etree
from lxml import etree
from lxml.etree import XMLSyntaxError
import pandas as pd
import numpy as np


In [58]:
PATH: str = '../data/'
FILES: str = '*.xml'
REGISTER: str = 'register.xml'
NAMESPACE: dict = {'tei': 'http://www.tei-c.org/ns/1.0'}
GND_REGEX: str = 'http://d-nb.info/gnd/'
GND_REGEX_ALT: str = 'https://d-nb.info/gnd/'


In [235]:
def add_to_register(ref: str, person_information_dict: dict, element):
    person_information_dict[ref] = {'name': element.text,
                                    'key': element.get('key')
                                    }
    return person_information_dict

def extract_metadata(file_path: str = PATH + FILES) -> (dict, dict):
    # Function for retrieving the needed metadata from the XML/TEI files.

    # A look-up table for all relevant information about the people in the corpus.
    person_information: dict = {}
    correspondences: dict = {}

    for file in glob.glob(file_path)[:]:
        try:
            # print(f"Working on {os.path.basename(file)}")
            tree = etree.parse(file)
            root = tree.getroot()

            sender_ref: str = ""
            receiver_ref: str = ""

            for elem in root.findall('./tei:teiHeader//tei:correspAction[@type="sent"]//tei:persName',
                                     NAMESPACE
                                     ):

                # Building the reference list for all persons existing in the corpus.
                sender_ref = (re.sub(GND_REGEX,
                                     '',
                                     str(elem.get('ref')),
                                     ).strip()
                              )

                if not sender_ref in person_information.keys():
                    person_information = add_to_register(sender_ref, person_information_dict=person_information, element=elem)


                """Adding sender to correspondences. At the beginning, the correspondence list will be empty and the first sender can be added. After that, they will only be added if their GND-key is not already in the correspondence list.
                """
                if not correspondences:
                    correspondences[sender_ref] = {}
                else:
                    if sender_ref not in correspondences.keys():
                        correspondences[sender_ref] = {}
                    else:
                        continue


            for elem in root.findall('./tei:teiHeader//tei:correspAction[@type="received"]//tei:persName',
                                     NAMESPACE
                                     ):
                receiver_ref = (re.sub(GND_REGEX,
                                       '',
                                       str(elem.get('ref'))
                                       ).strip()
                                )

                if not receiver_ref in person_information.keys():
                    person_information = add_to_register(receiver_ref, person_information_dict=person_information, element=elem)


            """Adding receivers to the list. This means that each person who sent a letter to someone will have that 'receiver' stored in a dictionary with the frequency of correspondance.
            """
            try:
                if not correspondences[sender_ref] or (correspondences[sender_ref] and receiver_ref not in correspondences[sender_ref].keys()):

                    correspondences[sender_ref][receiver_ref] = 1
                elif correspondences[sender_ref] and receiver_ref in correspondences[sender_ref].keys():
                    correspondences[sender_ref][receiver_ref] += 1
            except KeyError:
                # Entries with missing gnd-entries will be ignored as they can not be properly addressed.
                continue



        except XMLSyntaxError:
            # If there is a syntax error in the file, it will be ignored in the preprocessing.
            print(f"There has been a syntax error in {file}.")
            continue

    # Checking if the result is actually a set of only unique gnd-keys.
    # pprint.pprint(person_information.keys() == set(person_information.keys()))

    return correspondences, person_information




In [246]:
correspondences_, person_information_ = extract_metadata()

correspondences_df = pd.DataFrame.from_dict(correspondences_)
correspondences_df = correspondences_df.replace(np.nan, 0)
person_information_df = pd.DataFrame.from_dict(person_information_).transpose()

# new_directory: str = '../../data/retrieved/'
#
# if not os.path.exists(new_directory):
#     os.makedirs(new_directory)

correspondences_df.to_csv('correspondences.csv')
person_information_df.to_csv('person_information.csv')



There has been a syntax error in ../data/AWS-aw-02on.xml.
There has been a syntax error in ../data/AWS-aw-05q8.xml.
There has been a syntax error in ../data/AWS-aw-0317.xml.
There has been a syntax error in ../data/AWS-aw-05di.xml.
There has been a syntax error in ../data/AWS-aw-02wy.xml.
There has been a syntax error in ../data/AWS-aw-02le.xml.
There has been a syntax error in ../data/AWS-aw-031g.xml.
There has been a syntax error in ../data/AWS-aw-05x5.xml.
There has been a syntax error in ../data/AWS-aw-01b6.xml.
There has been a syntax error in ../data/AWS-aw-05uo.xml.
There has been a syntax error in ../data/AWS-aw-01ml.xml.
There has been a syntax error in ../data/AWS-aw-01b5.xml.
There has been a syntax error in ../data/AWS-aw-03ln.xml.
There has been a syntax error in ../data/AWS-aw-03dw.xml.
There has been a syntax error in ../data/AWS-aw-02xm.xml.
There has been a syntax error in ../data/AWS-aw-028q.xml.
There has been a syntax error in ../data/AWS-aw-02tj.xml.
There has been

In [261]:
"""Loading data with labels (names of the people) into a corresponding table for later import into Gephi. For visualisation purposes, it is important to include the labels in the dataset to be able to identify the nodes in the graph. Not all the necessary information is stored in the register.xml files, so instead, I created another person register which will be used to further enrich the data with fitting labels (names).
"""
def replace_ids(pds_df: pd.DataFrame, element):
    """
    Function for replacing the ids in the dataset with labels according to their gnd-key.

    :param pds_df: Pandas Dataframe that is used.
    :param element: lxml.etree.Element that is used to parse information from the XML file(s).
    :return: Updated pandas Dataframe.
    """
    pds_df.rename(columns={re.sub(GND_REGEX_ALT, '', element.attrib['ref']): element.text}, inplace=True)
    pds_df.loc[pds_df['Unnamed: 0'] == re.sub(GND_REGEX_ALT, '', element.attrib['ref']), 'Unnamed: 0'] = element.text

    return pds_df

def match_register(path: str = PATH + REGISTER):
    with open(path) as file:
        tree = etree.parse(file)
        root = tree.getroot()

        df = pd.read_csv('../data/retrieved/correspondences.csv')
        person_information_ = pd.read_csv('../data/retrieved/person_information.csv',
                                          usecols=['Unnamed: 0', 'name']
                                          ).set_index('Unnamed: 0')

        # For people who sent letters.
        for entry in df['Unnamed: 0']:
            #print('.//tei:persName[@ref="' + GND_REGEX_ALT + entry + '"]')
            for elem in root.findall('.//tei:persName[@ref="' + GND_REGEX_ALT + entry + '"]', NAMESPACE):
                df = replace_ids(pds_df=df, element=elem)


        # For people who are only listed as receivers (meaning in the column of the dataframe).
        for column in df.columns:
            if column == 'Unnamed: 0':
                continue
            else:
                for elem in root.findall('.//tei:persName[@ref="' + GND_REGEX_ALT + column + '"]', NAMESPACE):
                 df = replace_ids(pds_df=df, element=elem)

        # For people who are not listed in the register.xml file but who nevertheless have an entry in the person information file.
        person_information_ = person_information_.to_dict('index')
        for key, entry in person_information_.items():
            df = df.rename(columns={key: entry['name']})
            df.loc[df['Unnamed: 0'] == key, 'Unnamed: 0'] = entry['name']

        df = df.set_index('Unnamed: 0')
        df.to_csv('correspondences_with_names.csv')



In [262]:
match_register()